In [1]:
# multiprocess data from filename pool
%gui qt
from mptracker import MPTracker,TiffFileSequence
from mptracker import ellipseToContour
from json import load as jload

filename = '/home/joao/data/eyecam/181112_JC086_2P_JC/run00_gray/20181112_run000_00000000.tif'
parameterfile = '/home/joao/data/eyecam/181112_JC086_2P_JC/run00_gray/20181112_run000.json'

seq = TiffFileSequence(filename)

with open(parameterfile,'r') as fd:
    trackerpar = jload(fd)

from mptracker.parutils import par_process_tiff

res = par_process_tiff(seq.filenames,trackerpar)

/home/joao/data/eyecam/181112_JC086_2P_JC/run00_gray
72
There are 18229 frames in 72 files
	 - Processing 72 files.
	 - Analysed 18229 frames in 5.9 s [3075 fps]


In [6]:
import cv2
import numpy as np
class play_mptracker_results(object):
    paused = False
    gaussBlurSize = 3
    gaussBlurSigma = 2
    clahe = cv2.createCLAHE(clipLimit=35.0, tileGridSize=(8,8))
    def __init__(self,stack,results):
        self.movie = stack
        self.N = stack.nFrames
        self.results = results
        self.iFrame = 0
        self.wname = 'mptracker results'
        
    def play(self):
        cv2.namedWindow(self.wname)
        cv2.createTrackbar('Frame number',self.wname,0,self.N-1,lambda x:x)
        while True:
            if not self.paused:
                self.iFrame = np.mod(self.iFrame + 1,self.N)
                cv2.setTrackbarPos('Frame number',self.wname,self.iFrame)
            self.iFrame = cv2.getTrackbarPos('Frame number',self.wname)
            self.setImage(self.iFrame)
            k = cv2.waitKey(3)
            if k == ord('q'):
                cv2.destroyAllWindows()        
                break
            elif k == ord('s'):
                self.paused = not self.paused
        
    def setImage(self,iFrame):
    
        img = self.movie.get(self.iFrame)
        img = self.clahe.apply(img)
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
        
        cv2.putText(img, 'frame {0}'.format(self.iFrame), (50,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        1.0, (128, 128, 200), 
                        lineType=cv2.LINE_AA)
        if not np.isnan(self.results[self.iFrame,1][0]):
            xy = (int(res[self.iFrame,1][0]),int(res[self.iFrame,1][1]))
            c = ellipseToContour([xy[1],xy[0]],
                                 self.results[self.iFrame,3][0]/2.,
                                 self.results[self.iFrame,3][1]/2.,
                                 0)#res[self.iFrame,3][2]+90)
            cv2.drawContours(img,[c],0,[0,0,255],1)

            img = cv2.circle(img,xy , 2, [128,0,128], 1)
        
        cv2.imshow('mptracker results',img)
    
sbxplay = play_mptracker_results(seq,res)
sbxplay.play()


In [38]:
# With enhanced
class play_mptracker_results(object):
    gaussBlurSize = 3
    gaussBlurSigma = 2
    clahe = cv2.createCLAHE(clipLimit=35.0, tileGridSize=(8,8))
    paused = False
    def __init__(self,data,res):
        self.data = data
        self.res = res
        self.N = data.nFrames
        self.iFrame = 0
        self.lastImg = self.enhanceImage(self.data.get(self.iFrame))
        self.ellipse = np.vstack(res[:,3])
        self.fig = plt.figure()
        self.diamplt = plt.plot(self.ellipse[:,0])
        self.marker = plt.plot(self.iFrame,self.ellipse[self.iFrame,0],'o',markeredgecolor='r',markersize = 10)
    def enhanceImage(self,img):
        return self.clahe.apply(img) 
    
    def play(self):
        while True:
            img = self.enhanceImage(self.data.get(self.iFrame)) 
            self.lastImg = img.copy()
            cv2.putText(img, 'frame {0}'.format(self.iFrame), (50,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        1.0, (255, 255, 255), 
                        lineType=cv2.LINE_AA)
            if not np.isnan(res[self.iFrame,1][0]):
                xy = (int(res[self.iFrame,1][0]),int(res[self.iFrame,1][1]))
#                 img = getEllipseMask(img.shape,[xy[0],xy[1]],[res[self.iFrame,3][0]/2,
#                                      res[self.iFrame,3][1]/2],[res[self.iFrame,3][2]])
                c = ellipseToContour([xy[1],xy[0]],
                                     res[self.iFrame,3][0]/2.,
                                     res[self.iFrame,3][1]/2.,
                                     0)#res[self.iFrame,3][2]+90)
                img = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
                cv2.drawContours(img,[c],0,[0,0,255],1)
            img = cv2.circle(img,xy , 2, 255, 1)
            cv2.imshow('img',img)
            self.marker[0].set_xdata(self.iFrame)
            self.marker[0].set_ydata(self.ellipse[self.iFrame,0])
            plt.draw()
            plt.pause(0.01)
            if not self.paused:
                self.iFrame = np.mod(self.iFrame + 1,self.N)
            k = cv2.waitKey(3)
            if k == ord('q'):
                break
            elif k == ord('s'):
                self.paused = not self.paused
        cv2.destroyAllWindows()

splay = play_mptracker_results(seq,res)
splay.play()
#splay.save('/home/joao/data/facecam/whisker_top_view/topview_example.avi')

KeyboardInterrupt: 

In [7]:
%matplotlib qt
import pylab as plt


In [29]:
splay.marker.set_offsets?



In [10]:
plt.figure()
plt.plot()

In [3]:
# plot overlay
import cv2
import numpy as np
class playStack(object):
    gaussBlurSize = 3
    gaussBlurSigma = 2
    clahe = cv2.createCLAHE(clipLimit=35.0, tileGridSize=(8,8))
    paused = False
    def __init__(self,data,res):
        self.data = data
        self.res = res
        self.N = data.nFrames
        self.iFrame = 0
        self.lastImg = self.enhanceImage(self.data.get(self.iFrame))
    def enhanceImage(self,img):
        return self.clahe.apply(cv2.GaussianBlur(img,
                                                 (self.gaussBlurSize,
                                                  self.gaussBlurSize),
                                                 self.gaussBlurSigma))   
    def play(self):
        while True:
            img = self.enhanceImage(self.data.get(self.iFrame)) 
            self.lastImg = img.copy()
            cv2.putText(img, 'frame {0}'.format(self.iFrame), (50,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        1.0, (255, 255, 255), 
                        lineType=cv2.LINE_AA)
            if not np.isnan(res[self.iFrame,1][0]):
                xy = (int(res[self.iFrame,1][0]),int(res[self.iFrame,1][1]))
            
                c = ellipseToContour([xy[1],xy[0]],
                                     res[self.iFrame,3][0]/2.,
                                     res[self.iFrame,3][1]/2.,
                                     0)#res[self.iFrame,3][2]+90)

                cv2.drawContours(img,[c],0,255,1)
            img = cv2.circle(img,xy , 2, 255, 1)
            cv2.imshow('img',img)
            if not self.paused:
                self.iFrame = np.mod(self.iFrame + 1,self.N)
            k = cv2.waitKey(3)
            if k == ord('q'):
                break
            elif k == ord('s'):
                self.paused = not self.paused
        cv2.destroyAllWindows()

splay = playStack(seq,res)
splay.play()
#splay.save('/home/joao/data/facecam/whisker_top_view/topview_example.avi')

In [3]:
from mptracker import ellipseToContour
import pylab as plt
%matplotlib qt
c = ellipseToContour(res[0,1],res[0,3][0]/2,res[0,3][1]/2,res[0,3][2])
plt.imshow(seq.get(0)**0.5,cmap = 'gray')
plt.plot(c[:,0,1],c[:,0,0],'y')

In [59]:
from mptracker import convertPixelToEyeCoords
az,el,theta = convertPixelToEyeCoords(np.hstack(res[1]),
                                      (trackerpar['points'][0],trackerpar['points'][2]),
                                      np.hstack(res[0]))

IndexError: too many indices for array

In [6]:
from mptracker import getEllipseMask
MPTracker.getAugmented??